In [1]:
from csv import reader

In [2]:
def load_csv(filename):
    dataset = list()
    with open(filename, "r") as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            r2 = []
            for i in range(len(row)):
                r2.append(row[i][0].lower())
            dataset.append(r2)
    return dataset

In [3]:
training_data = load_csv("DataSet3.csv")

In [4]:
header = ["Birth", "Fly", "Water", "Legs", "Wings"]

In [5]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [6]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [7]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [8]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))

In [9]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [10]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl ** 2
    return impurity

In [11]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [12]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1
    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question

In [13]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [14]:
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [15]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

In [16]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    print(spacing + str(node.question))
    print(spacing + "--> True:")
    print_tree(node.true_branch, spacing + "  ")
    print(spacing + "--> False:")
    print_tree(node.false_branch, spacing + "  ")

In [17]:
my_tree = build_tree(training_data)

In [18]:
print_tree(my_tree)

Is Birth == n?
--> True:
  Is Water == n?
  --> True:
    Is Wings == n?
    --> True:
      Is Legs == n?
      --> True:
        Predict {'n': 1}
      --> False:
        Predict {'n': 2, 'm': 1}
    --> False:
      Predict {'n': 2}
  --> False:
    Predict {'n': 6}
--> False:
  Is Legs == n?
  --> True:
    Predict {'m': 1, 'n': 1}
  --> False:
    Predict {'m': 4}


In [19]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [20]:
classify(training_data[0], my_tree)

{'m': 4}

In [21]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [22]:
print_leaf(classify(training_data[0], my_tree))

{'m': '100%'}

In [23]:
print_leaf(classify(training_data[1], my_tree))

{'n': '100%'}

In [24]:
testing_data = [
    ["y", "n", "y", "n", "y"],
]

In [25]:
for row in testing_data:
    print("Predicted: %s" % (print_leaf(classify(row, my_tree))))

Predicted: {'m': '50%', 'n': '50%'}
